<a href="https://colab.research.google.com/github/planchon/tweeter_fakenews/blob/main/Tweeter_%26_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tweeter & Fake news
Notebook jupyter du projet tweeter et fake news par Arnaud Valette & Paul Planchon

run the first cell once to be sure all the librairies are installed

In [ ]:
!pip3 install tweepy pandas numpy matplotlib python-dotenv

In [ ]:
import tweepy
import pickle
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dotenv import dotenv_values
import requests

### env vars
in a folder `../creds` put an .env file with
```
API_KEY=""
API_SECRET=""
API_TOKEN=""
```
which are the creds from twitter dev portal [twitter portal](https://developer.twitter.com/en/portal/projects/)

In [ ]:
env_var = dotenv_values("../creds/.env")
API_KEY    = env_var["API_KEY"]
API_SECRET = env_var["API_SECRET"]
API_TOKEN  = env_var["API_TOKEN"]

In [ ]:
auth = tweepy.AppAuthHandler(API_KEY, API_SECRET)
api = tweepy.API(auth)

## Monde website data
getting the JSON data from [le monde](https://www.lemonde.fr/webservice/decodex/updates) decodex updates feed

In [ ]:
try:
    decodex_site = pd.read_csv("./monde_site.csv", index_col=0)
    with open('monde_urls.pickle', 'rb') as handle:
        site_index_to_url = pickle.load(handle)
except Exception:
    decodex_site = pd.DataFrame(requests.get("https://www.lemonde.fr/webservice/decodex/updates").json()["sites"]).T.drop(3, 1).rename(columns={0: "categorie", 1: "description", 2: "site"})
    decodex_url = requests.get("https://www.lemonde.fr/webservice/decodex/updates").json()["urls"] # urls of website, including sometimes twitter data

    # convert the url to index dict to an index to url dict
    site_index_to_url = {}
    for x, y in decodex_url.items():
        try:
            if site_index_to_url[y]:
                site_index_to_url[y].append(x)
        except Exception as e:
            site_index_to_url[y] = [x]

    # we save data to disk
    decodex_site.to_csv("monde_site.csv")
    with open('monde_urls.pickle', 'wb') as handle:
        pickle.dump(site_index_to_url, handle, protocol=pickle.HIGHEST_PROTOCOL)